# Splitting Data for Gender Bias Token Classifiers

* **Supervised learning**
    * Source data: `../data/token_clf_data/`
    * Output data: train, validate (a.k.a. development), and test splits under `..data/token_clf_data/model_input/`
* **Multilabel classification**
    * 3 categories of labels:
        1. *Person Name:* Unknown, Non-binary,* Feminine, Masculine
        2. *Linguistic:* Generalization, Gendered Pronoun, Gendered Role
        3. *Contextual:* Empowering,* Occupation, Omission, Stereotype

*Annotators did not find text on which to apply these labels during the manual annotation process!

***

**Table of Contents**

[0. Setup](#0)

[1. Preprocess Data](#1)

[2. Summarize the Data](#2)

[3. Split the Data](#3)


***

<a id="0"></a>

## 0. Setup

**Import libraries and load data**

In [1]:
# For custom functions and for paths
import utils, config

# For working with data files and directories
import numpy as np
import pandas as pd
from pathlib import Path
import os, re

# For preprocessing the text
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
# nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.tag import pos_tag

# For comparing text offsets
from intervaltree import Interval, IntervalTree

Load the data:

In [2]:
df_tags = pd.read_csv(config.tokc_path+"tagged_tokens.csv", index_col=0) #
df_tags.head()

,ann_id,description_id,tag,token_id
0,NaN,0,O,0
0,NaN,0,O,1
0,NaN,0,O,2
0,14384.0,1,B-Unknown,7
0,24275.0,1,B-Masculine,7


In [3]:
df_dsat = pd.read_csv(config.tokc_path+"desc_sent_ann_token_tag.csv")
subdf_dsat = df_dsat[["token_id", "sentence_id", "description_id", "token", "token_offsets"]]
subdf_dsat.head()

,token_id,sentence_id,description_id,token,token_offsets
0,0,0,0,Identifier,"(0, 10)"
1,1,0,0,:,"(10, 11)"
2,2,0,0,AA5,"(12, 15)"
3,3,1,1,Title,"(17, 22)"
4,4,1,1,:,"(22, 23)"


In [24]:
subdf_dsat.loc[subdf_dsat.sentence_id == 4]

,token_id,sentence_id,description_id,token,token_offsets
134,134,4,3,He,"(789, 791)"
135,135,4,3,was,"(792, 795)"
136,136,4,3,educated,"(796, 804)"
137,137,4,3,at,"(805, 807)"
138,138,4,3,Daniel,"(808, 814)"
139,139,4,3,Stewart,"(815, 822)"
140,140,4,3,'s,"(822, 824)"
141,141,4,3,College,"(825, 832)"
142,142,4,3,and,"(833, 836)"
143,143,4,3,the,"(837, 840)"


## 2. Preprocess the Data

Join the data on the token IDs.

In [27]:
df = df_tags.join(subdf_dsat.set_index(["token_id", "description_id"]), on=["token_id", "description_id"], how="outer")
df.head()

,ann_id,description_id,tag,token_id,sentence_id,token,token_offsets
0,NaN,0,O,0,0,Identifier,"(0, 10)"
0,NaN,0,O,1,0,:,"(10, 11)"
0,NaN,0,O,2,0,AA5,"(12, 15)"
0,14384.0,1,B-Unknown,7,1,The,"(34, 37)"
0,24275.0,1,B-Masculine,7,1,The,"(34, 37)"


In [31]:
# Replace NaN values in ann_id column with 99999 to indicate rows without an annotation
df[["ann_id"]] = df["ann_id"].fillna("99999")
df = df.astype({"ann_id":"int32"})
# Replace NaN values in tag column with O to indicate rows without an annotation
df[["tag"]] = df["tag"].fillna("O")
df.loc[df.sentence_id == 4].head()

,ann_id,description_id,tag,token_id,sentence_id,token,token_offsets
1,14377,3,B-Gendered-Pronoun,134,4,He,"(789, 791)"
1,14378,3,B-Gendered-Pronoun,148,4,he,"(871, 873)"
14778,99999,3,O,135,4,was,"(792, 795)"
14778,99999,3,O,136,4,educated,"(796, 804)"
14778,99999,3,O,137,4,at,"(805, 807)"


In [32]:
df = df[["description_id", "sentence_id", "ann_id", "token_id", "token", "tag"]]
df = df.sort_values(by=["description_id", "sentence_id", "token_id"])
df.head()

,description_id,sentence_id,ann_id,token_id,token,tag
0,0,0,99999,0,Identifier,O
0,0,0,99999,1,:,O
0,0,0,99999,2,AA5,O
14778,1,1,99999,3,Title,O
14778,1,1,99999,4,:,O


In [40]:
df.loc[df.token.isna()].shape

(0, 6)

<a id="2"></a>
## 2. Summarize the Data

In [45]:
print("Total descriptions:", len(df.description_id.unique()))
print("Total sentences:", len(df.sentence_id.unique()))
print("Total annotations:", len(df.ann_id.unique())-1)  # -1 because 9999 value indicates no annotation
print("Total tokens:", len(df.token_id.unique()))
print("Total token tags:", df.shape[0])

Total descriptions: 27908
Total sentences: 42030
Total annotations: 55218
Total tokens: 753932
Total token tags: 784542


In [47]:
unique_tokens = list(set(df.token))
unique_words = [token for token in unique_tokens if token.isalpha()]
print("Total word types (unique words):", len(unique_words))

Total word types (unique words): 36462


In [48]:
# Reference: https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
df_tag_totals = df.groupby('tag').size().reset_index(name='total')
df_tag_totals = df_tag_totals.sort_values(by="total")
df_tag_totals

,tag,total
15,I-Nonbinary,1
5,B-Nonbinary,1
11,I-Gendered-Pronoun,67
12,I-Gendered-Role,726
13,I-Generalization,891
0,B-Feminine,1614
3,B-Generalization,2051
8,B-Stereotype,2614
2,B-Gendered-Role,3577
10,I-Feminine,3782


In [49]:
df_tag_totals.to_csv(config.tokc_path+"token_tag_totals.csv")

In [50]:
# # Make the token_offsets column values tuples of ints
# token_offsets = list(df_tags.token_offsets)
# token_offsets_clean = [offsets[1:-1].split(", ") for offsets in token_offsets if type(offsets)]
# token_offsets_tuples = [tuple((int(offsets[0]), int(offsets[1]))) for offsets in token_offsets_clean]
# df_tags = df_tags.drop(columns=["token_offsets"])
# df_tags.insert(len(df_tags.columns), "token_offsets", token_offsets_tuples)

Save the data for token classification:

In [51]:
Path(config.tokc_path+"model_input").mkdir(parents=True, exist_ok=True)

In [52]:
df = df.reset_index()
df = df.drop(columns="index")
df.head()

,description_id,sentence_id,ann_id,token_id,token,tag
0,0,0,99999,0,Identifier,O
1,0,0,99999,1,:,O
2,0,0,99999,2,AA5,O
3,1,1,99999,3,Title,O
4,1,1,99999,4,:,O


In [53]:
df.to_csv(config.tokc_path+"model_input/all_token_data.csv")

<a id="3"></a>
## 3. Split the Data

Split the data randomly, balancing the number of metadata field types (Title, Scope and Contents, Biographical / Historical, and Processing Information) across the train, validation, and test splits.

Implode the data,  grouping tokens by their sentence, then explode the data after the sentences are split into train, validation, and test sets so each row has one token.

In [54]:
df_descs = pd.read_csv(config.crc_meta_path+"annot_descs.csv")
df_descs = df_descs[["description_id", "field"]]
df_descs.head()

,description_id,field
0,0,Identifier
1,1,Title
2,2,Scope and Contents
3,3,Biographical / Historical
4,4,Identifier


In [55]:
df = pd.read_csv(config.tokc_path+"model_input/all_token_data.csv", index_col=0)
df = df.join(df_descs.set_index("description_id"), on="description_id", how="left")
df.head()

,description_id,sentence_id,ann_id,token_id,token,tag,field
0,0,0,99999,0,Identifier,O,Identifier
1,0,0,99999,1,:,O,Identifier
2,0,0,99999,2,AA5,O,Identifier
3,1,1,99999,3,Title,O,Title
4,1,1,99999,4,:,O,Title


In [56]:
# ann_id_values = list(df.loc[~df.ann_id.isna()].ann_id)
# print(max(ann_id_values))  # 55259.0

Shuffle the sentence IDs randomly and then assign them to train, validation, and (blind) test set splits of the data for creating and evaluating token classifiers:

In [57]:
sents = df[["sentence_id"]]
sents = sents.drop_duplicates()
shuffled_sents = utils.shuffleDataFrame(sents)
# shuffled_sents.head()
train_size, validat_size, test_size = utils.getTrainValTestSizes(shuffled_sents)
shuffled_sents_splits = utils.assignSubsets(shuffled_sents, train_size, validat_size, test_size)
shuffled_sents_splits.head()

,subset,sentence_id
780575,train,41770
184904,train,7756
347141,train,15662
47948,train,2041
599248,train,31629


In [58]:
shuffled_sents_splits.loc[shuffled_sents_splits.sentence_id == 5]

,subset,sentence_id
172,dev,5


In [59]:
shuffled_sents_splits.groupby("subset").size().reset_index(name="subset_count")  # Looks good

,subset,subset_count
0,dev,8406
1,test,8406
2,train,25218


Join the subset data with the token data:

In [60]:
df_joined = df.join(shuffled_sents_splits.set_index("sentence_id"), on="sentence_id", how="left")
df_joined.head()

,description_id,sentence_id,ann_id,token_id,token,tag,field,subset
0,0,0,99999,0,Identifier,O,Identifier,test
1,0,0,99999,1,:,O,Identifier,test
2,0,0,99999,2,AA5,O,Identifier,test
3,1,1,99999,3,Title,O,Title,train
4,1,1,99999,4,:,O,Title,train


In [61]:
df_joined.loc[df_joined.sentence_id == 4]

,description_id,sentence_id,ann_id,token_id,token,tag,field,subset
152,3,4,14377,134,He,B-Gendered-Pronoun,Biographical / Historical,test
153,3,4,99999,135,was,O,Biographical / Historical,test
154,3,4,99999,136,educated,O,Biographical / Historical,test
155,3,4,99999,137,at,O,Biographical / Historical,test
156,3,4,99999,138,Daniel,O,Biographical / Historical,test
157,3,4,99999,139,Stewart,O,Biographical / Historical,test
158,3,4,99999,140,'s,O,Biographical / Historical,test
159,3,4,99999,141,College,O,Biographical / Historical,test
160,3,4,99999,142,and,O,Biographical / Historical,test
161,3,4,99999,143,the,O,Biographical / Historical,test


Looks good!

**Train Data:**

In [62]:
df_train = df_joined.loc[df_joined.subset == "train"]
print(df_train.shape)
df_train.head()

(470712, 8)


,description_id,sentence_id,ann_id,token_id,token,tag,field,subset
3,1,1,99999,3,Title,O,Title,train
4,1,1,99999,4,:,O,Title,train
5,1,1,99999,5,Papers,O,Title,train
6,1,1,99999,6,of,O,Title,train
7,1,1,14384,7,The,B-Unknown,Title,train


In [63]:
# Reference: https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
df_tag_totals = df_train.groupby('tag').size().reset_index(name='total')
df_tag_totals = df_tag_totals.sort_values(by="total")
df_tag_totals = df_tag_totals.reset_index()
df_tag_totals.drop("index", axis=1)

,tag,total
0,I-Nonbinary,1
1,B-Nonbinary,1
2,I-Gendered-Pronoun,31
3,I-Gendered-Role,434
4,I-Generalization,528
5,B-Feminine,927
6,B-Generalization,1280
7,B-Stereotype,1595
8,I-Feminine,2051
9,B-Gendered-Role,2146


**Validation Data:**

In [64]:
df_validate = df_joined.loc[df_joined.subset == "dev"]
print(df_validate.shape)
df_validate.head()

(158836, 8)


,description_id,sentence_id,ann_id,token_id,token,tag,field,subset
172,3,5,99999,154,After,O,Biographical / Historical,dev
173,3,5,14379,155,his,B-Gendered-Pronoun,Biographical / Historical,dev
174,3,5,99999,156,ordination,O,Biographical / Historical,dev
175,3,5,14380,157,he,B-Gendered-Pronoun,Biographical / Historical,dev
176,3,5,99999,158,spent,O,Biographical / Historical,dev


In [65]:
# Reference: https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
df_tag_totals = df_validate.groupby('tag').size().reset_index(name='total')
df_tag_totals = df_tag_totals.sort_values(by="total")
df_tag_totals = df_tag_totals.reset_index()
df_tag_totals.drop("index", axis=1)

,tag,total
0,I-Gendered-Pronoun,18
1,I-Gendered-Role,158
2,I-Generalization,165
3,B-Feminine,357
4,B-Generalization,386
5,B-Stereotype,527
6,B-Gendered-Role,767
7,B-Gendered-Pronoun,833
8,B-Occupation,867
9,I-Feminine,949


**Test Data:**

In [66]:
df_test = df_joined.loc[df_joined.subset == "test"]
print(df_test.shape)
df_test.head()

(154994, 8)


,description_id,sentence_id,ann_id,token_id,token,tag,field,subset
0,0,0,99999,0,Identifier,O,Identifier,test
1,0,0,99999,1,:,O,Identifier,test
2,0,0,99999,2,AA5,O,Identifier,test
152,3,4,14377,134,He,B-Gendered-Pronoun,Biographical / Historical,test
153,3,4,99999,135,was,O,Biographical / Historical,test


In [67]:
# Reference: https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
df_tag_totals = df_test.groupby('tag').size().reset_index(name='total')
df_tag_totals = df_tag_totals.sort_values(by="total")
df_tag_totals = df_tag_totals.reset_index()
df_tag_totals.drop("index", axis=1)

,tag,total
0,I-Gendered-Pronoun,18
1,I-Gendered-Role,134
2,I-Generalization,198
3,B-Feminine,330
4,B-Generalization,385
5,B-Stereotype,492
6,B-Gendered-Role,664
7,B-Occupation,688
8,I-Occupation,766
9,I-Feminine,782


Write the data splits to a file:

In [68]:
df_train.to_csv(config.tokc_path+"model_input/token_train.csv")
df_validate.to_csv(config.tokc_path+"model_input/token_validate.csv")
df_test.to_csv(config.tokc_path+"model_input/token_test.csv")